# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `01_materials/labs/2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [19]:
# Write your code below.
%load_ext dotenv
%dotenv 
PRICE_DATA = os.getenv("PRICE_DATA")

print(PRICE_DATA)



The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv
../../05_src/data/prices/


In [20]:
import dask.dataframe as dd

+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [ ]:
import os
from glob import glob

# Write your code below
price_data_ST = glob(os.path.join(os.getenv('PRICE_DATA'), "*/*/*.parquet"))
price_data_ST





['../../05_src/data/prices\\ACN\\ACN_2001\\part.0.parquet',
 '../../05_src/data/prices\\ACN\\ACN_2001\\part.1.parquet',
 '../../05_src/data/prices\\ACN\\ACN_2002\\part.0.parquet',
 '../../05_src/data/prices\\ACN\\ACN_2002\\part.1.parquet',
 '../../05_src/data/prices\\ACN\\ACN_2003\\part.0.parquet',
 '../../05_src/data/prices\\ACN\\ACN_2003\\part.1.parquet',
 '../../05_src/data/prices\\ACN\\ACN_2004\\part.0.parquet',
 '../../05_src/data/prices\\ACN\\ACN_2004\\part.1.parquet',
 '../../05_src/data/prices\\ACN\\ACN_2005\\part.0.parquet',
 '../../05_src/data/prices\\ACN\\ACN_2005\\part.1.parquet',
 '../../05_src/data/prices\\ACN\\ACN_2006\\part.0.parquet',
 '../../05_src/data/prices\\ACN\\ACN_2006\\part.1.parquet',
 '../../05_src/data/prices\\ACN\\ACN_2007\\part.0.parquet',
 '../../05_src/data/prices\\ACN\\ACN_2007\\part.1.parquet',
 '../../05_src/data/prices\\ACN\\ACN_2008\\part.0.parquet',
 '../../05_src/data/prices\\ACN\\ACN_2008\\part.1.parquet',
 '../../05_src/data/prices\\ACN\\ACN_200

In [40]:
dd_px = dd.read_parquet(price_data_ST)

For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Close:
    
    - `returns`: (Close / Close_lag_1) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [31]:
%run update_path.py

In [33]:
src_path

WindowsPath('C:/Users/Ecce H/Dir5/production/05_src')

In [51]:

dd_function = (
    dd_px
        .groupby('ticker', group_keys=False)
        .apply(
            lambda x: x.sort_values('Date', ascending = True)
                       .assign(Adj_Close_lag_1 = x['Adj Close'].shift(1), Close_lag_1 = x['Close'].shift(1))
                       .assign(Returns = lambda x: x['Close']/x['Close_lag_1'] - 1, 
                       hi_lo_range = lambda x : x['High']- x['Low']), 
            meta = pd.DataFrame(data ={'Date': 'datetime64[ns]',
                    'Open': 'f8',
                    'High': 'f8',
                    'Low': 'f8',
                    'Close': 'f8',
                    'Adj Close': 'f8',
                    'Volume': 'i8',
                    'source': 'object',
                    'ticker': 'object',
                    'Year': 'int32',
                    'Adj_Close_lag_1': 'f8',
                    'Close_lag_1': 'f8',
                    'Returns': 'f8',
                    'hi_lo_range': 'f8'},
                    index = pd.Index([], dtype=pd.StringDtype(), name='ticker'))
    ))

In [52]:
dt = dd_function.compute()

+ Convert the Dask data frame to a pandas data frame. 
+ Add a new feature containing the moving average of `returns` using a window of 10 days. There are several ways to solve this task, a simple one uses `.rolling(10).mean()`.

(3 pt)

In [ ]:

dt



,Date,Open,High,Low,Close,Adj Close,Volume,source,ticker,Year,Adj_Close_lag_1,Close_lag_1,Returns,hi_lo_range
0,2017-10-26,9.970000,9.970000,9.970000,9.970000,9.970000,0.0,EYE.csv,EYE,2017,37.450001,37.450001,-0.733778,0.000000
1,2017-10-27,27.690001,28.370001,27.379999,28.150000,28.150000,954400.0,EYE.csv,EYE,2017,9.970000,9.970000,1.823470,0.990002
2,2017-10-30,28.070000,28.900000,27.490000,28.040001,28.040001,760500.0,EYE.csv,EYE,2017,28.150000,28.150000,-0.003908,1.410000
3,2017-10-31,28.000000,29.389999,27.681000,28.799999,28.799999,436200.0,EYE.csv,EYE,2017,28.040001,28.040001,0.027104,1.709000
4,2017-11-01,29.049999,30.980000,28.270000,30.370001,30.370001,1781300.0,EYE.csv,EYE,2017,28.799999,28.799999,0.054514,2.709999
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
177923,2020-03-26,123.620003,138.440002,123.139999,137.050003,137.050003,1589800.0,LH.csv,LH,2020,122.820000,122.820000,0.115861,15.300003
177924,2020-03-27,131.869995,133.289993,126.360001,128.710007,128.710007,1058600.0,LH.csv,LH,2020,137.050003,137.050003,-0.060854,6.929993
177925,2020-03-30,128.000000,132.580002,123.349998,131.949997,131.949997,971200.0,LH.csv,LH,2020,128.710007,128.710007,0.025173,9.230003
177926,2020-03-31,130.750000,131.270004,123.059998,126.389999,126.389999,1273700.0,LH.csv,LH,2020,131.949997,131.949997,-0.042137,8.210007


In [58]:
dt.columns[dt.columns.duplicated()]

Index([], dtype='object')

In [60]:

dt = (dt.groupby('ticker', group_keys = False)
    .apply(lambda x: x.assign(ma_return = x['Returns'].rolling(10).mean())))

C:\Users\Ecce H\AppData\Local\Temp\ipykernel_3076\1946340975.py:2: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: x.assign(ma_return = x['Returns'].rolling(10).mean())))


Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?
+ Would it have been better to do it in Dask? Why?

(1 pt)

In [ ]:
No, it was not necessary to convert to pandas to calculate the moving average return
Yes, it would habe been better to do it in Dask as we were already working in Dask

## Criteria

The [rubric](./assignment_1_rubric_clean.xlsx) contains the criteria for grading.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.